# Fixing Negative Concentrations 

In [194]:
import numpy as np
import copy
import pprint

In [212]:
A      = np.array([ [ 0, 1,-1, 0, 0, 0, 0, 0, 0, 0],\
                  [ 1, 0,-1, 0, 0, 0,-1, 0, 0, 0],\
                  [-1, 0, 1, 0, 0, 0, 1,-1, 0, 0],\
                  [ 0, 0, 0,-1,-1,-1, 0, 0, 0, 0],\
                  [ 0, 0, 0, 2, 0, 1,-1, 0, 0, 1],\
                  [ 0, 0, 0, 0, 0, 0, 0, 0,-1,-1]])


### Step 1: Machine learning step,  let `CO` be the concentration input and `S` be the flux output

In [233]:
C0 = np.array([0.0445,    0.0001,   0.0007,    0.0002,   0.0001,    0.0059]);
S  = np.array([0.00070,    0.00070,   0.00091,  0, 0, 0, 0.00011, 0, 0, 0 ]);

### Step 2: Finding negative concentrations 
Multiplication by stoichiometric matrix to get change in concentration, ∆C = AS.
Then addition of `delC` to `C0` to get `C1`, the concentration of the next timestep.  If there are any negative values for `C1`, find the location of the most negative value.

In [237]:
delC = A@S
C1 = C0 + delC;
if any(C1 < 0):
    print(C1)
    loc = C1.argmin()
    print(f"The concentration at index {location} is negative!")

[ 4.429e-02 -2.200e-04  1.020e-03  2.000e-04 -1.000e-05  5.900e-03]
The concentration at index 1 is negative!


### Step 3: Shrinking the fluxes that led to negative concentrations

Though the negative element of `C1` may have been contributed to by multiple elements of `S`, we only want to shrink the elements that caused a negative concentration -- in other words, the location of all the negative column elements of `A`, at the row corresponding to the negative element of `C1`.  We want to adjust these "depleting" elements of `S` while leaving the other elements alone. 

In [238]:
S_corrected = copy.deepcopy(S);
C_corrected = copy.deepcopy(C1)

N = S[np.where(A[loc,:]<0)[0]]
M = S[np.where(A[loc,:]>0)[0]]

for n in np.where(A[loc,:]<0)[0]:
    S_corrected[n] = (1 - (delC[loc]+C0[loc])/sum(S[np.where(A[loc,:]<0)[0]])/ A[loc,n] ) * S_corrected[n]

print("Nonphysical ML prediction S:")
print(np.round(S,5))
print("Adjusted S vector")
print(np.round(S_corrected,5))


Nonphysical ML prediction S:
[0.0007  0.0007  0.00091 0.      0.      0.      0.00011 0.      0.
 0.     ]
Adjusted S vector
[7.0e-04 7.0e-04 7.1e-04 0.0e+00 0.0e+00 0.0e+00 9.0e-05 0.0e+00 0.0e+00
 0.0e+00]


### Step 4: Calculate the adjusted C

Use `S_corrected`  in ∆C = AS, and then add that to `C0`.

In [240]:
delC_corrected = A@S_corrected
C_corrected = C0 + A@S_corrected

print("Initial concentration C0:")
print(np.round(C0,5))
print("Nonphysical ML prediction C1:")
print(np.round(C1,5))
print("Adjusted concentration:")
print(np.round(C_corrected,5))

Initial concentration C0:
[0.0445 0.0001 0.0007 0.0002 0.0001 0.0059]
Nonphysical ML prediction C1:
[ 4.429e-02 -2.200e-04  1.020e-03  2.000e-04 -1.000e-05  5.900e-03]
Adjusted concentration:
[4.449e-02 0.000e+00 8.000e-04 2.000e-04 1.000e-05 5.900e-03]
